# Regressão Linear

## Introdução
A regressão linear é um dos modelos mais fundamentais em aprendizado de máquina supervisionado.  
Seu objetivo é **estimar uma variável de saída contínua** (alvo $y \in \mathbb{R}$) a partir de variáveis de entrada $x \in \mathbb{R}^d$.  

Formalmente, desejamos encontrar uma função linear da forma:

$$
h(x) = w_0 + w_1 x_1 + w_2 x_2 + \cdots + w_d x_d = w^T \tilde{x}
$$

onde:
- $\tilde{x} = (1, x_1, \dots, x_d)$ inclui o termo de bias,
- $w = (w_0, w_1, \dots, w_d)$ é o vetor de parâmetros a serem aprendidos.

---

## Função de perda
Para avaliar a qualidade da hipótese $h(x)$, utilizamos a **função de perda quadrática**:

$$
L_D(h) = \frac{1}{N} \sum_{n=1}^N \big(h(x^{(n)}) - y^{(n)} \big)^2
$$

onde:
- $(x^{(n)}, y^{(n)})$ são os exemplos do conjunto de treinamento,
- $N$ é o número total de exemplos.

A tarefa de aprendizado consiste em encontrar $w$ que minimize $L_D(h)$.

---

## Soluções
Existem diferentes estratégias para determinar o vetor de parâmetros $w$:

1. **Solução Analítica (Equações Normais)**  
   Quando $d$ e $N$ não são muito grandes, podemos resolver o sistema linear:
   $$
   w = (X^T X)^{-1} X^T y
   $$
   onde $X$ é a matriz de dados de dimensão $N \times (d+1)$.

2. **Solução Iterativa (Gradiente Descendente)**  
   Para conjuntos grandes, o cálculo da inversa pode ser inviável.  
   O gradiente descendente ajusta $w$ iterativamente:
   $$
   w^{(r+1)} = w^{(r)} - \eta \nabla L(w^{(r)})
   $$
   com taxa de aprendizado $\eta$ e gradiente:
   $$
   \frac{\partial L}{\partial w_j} = \sum_{n=1}^N (h(x^{(n)}) - y^{(n)}) x^{(n)}_j
   $$

3. **SGD (Stochastic Gradient Descent)**  
   Em vez de usar todos os exemplos a cada iteração, atualizamos os pesos a partir de **um único exemplo** (ou mini-lote).  
   Isso reduz custo computacional e introduz ruído que ajuda a escapar de mínimos locais.

---

## Avaliação da Regressão Linear
Uma métrica amplamente utilizada é o **coeficiente de determinação** ($R^2$):

$$
R^2 = 1 - \frac{SS_{res}}{SS_{tot}}
$$

onde:
- $SS_{res} = \sum_{n=1}^N (y^{(n)} - \hat{y}^{(n)})^2$ (soma dos resíduos),
- $SS_{tot} = \sum_{n=1}^N (y^{(n)} - \bar{y})^2$ (soma total das variâncias).

Valores próximos de 1 indicam bom ajuste; valores próximos de 0 ou negativos indicam ajuste ruim.

# Importando o dataset

In [14]:
import pandas as pd

# A função pd.read_csv() lê o arquivo e o carrega em um DataFrame do pandas.
df = pd.read_csv('data_0460_5832.csv')

# O comando .head() mostra as 5 primeiras linhas do DataFrame.
print("Primeiras 5 linhas do dataset:")
print(df.head())

# O comando .info() mostra um resumo técnico, incluindo os tipos de cada coluna.
print('\nInformações do DataFrame:')
df.info()

Primeiras 5 linhas do dataset:
      Sex  Age  Height  Weight  Shoe number
0  Female   53     154      59           36
1    Male   23     170      56           40
2  Female   23     167      63           37
3    Male   21     178      78           40
4  Female   25     153      58           36

Informações do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Sex          257 non-null    object
 1   Age          257 non-null    int64 
 2   Height       257 non-null    int64 
 3   Weight       257 non-null    int64 
 4   Shoe number  257 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 10.2+ KB


# Tratamento dos dados

In [26]:
# Copiar o df original para não sobrescrever
df_encoded = df.copy()

# Identificar colunas categóricas que só têm dois valores
for col in df_encoded.select_dtypes(include=['object', 'category']).columns:
    valores = df_encoded[col].dropna().unique()
    if len(valores) == 2:
        # mapear automaticamente para 0/1
        mapping = {valores[0]: 0, valores[1]: 1}
        df_encoded[col] = df_encoded[col].map(mapping)
        print(f"Coluna {col} mapeada para {mapping}")
    else:
        df_encoded.drop(columns=[col], inplace=True)
# remove colunas constantes
num_cols = df_encoded.select_dtypes(include=[np.number]).columns.tolist()
num_cols = [c for c in num_cols if df_encoded[c].nunique(dropna=True) > 1]


Coluna Sex mapeada para {'Female': 0, 'Male': 1}


# Visualização da distribuicao dos dados, par a par

In [27]:
import itertools
import matplotlib.pyplot as plt
import numpy as np

# Agora pega numéricas + as recém convertidas
num_cols = df_encoded.select_dtypes(include=[np.number]).columns.tolist()
print("Colunas numéricas (incluindo binárias codificadas):", num_cols)

# Gerar todas as combinações 2 a 2
pairs = list(itertools.combinations(num_cols, 2))

fig, axes = plt.subplots(len(pairs)//2 + len(pairs)%2, 2, figsize=(12, 5*len(pairs)//2))
axes = axes.flatten()

for ax, (xcol, ycol) in zip(axes, pairs):
    ax.scatter(df_encoded[xcol], df_encoded[ycol], alpha=0.6)
    ax.set_xlabel(xcol)
    ax.set_ylabel(ycol)
    ax.set_title(f"{xcol} vs {ycol}")

# ocultar plots vazios
for ax in axes[len(pairs):]:
    ax.set_visible(False)

plt.tight_layout()
plt.show()


Colunas numéricas (incluindo binárias codificadas): ['Sex', 'Age', 'Height', 'Weight', 'Shoe number']


### Regressão Linear Analítica

In [80]:
from types import SimpleNamespace

args = SimpleNamespace()
args.plot = False
args.top_k = 15

In [81]:
%run -i regressao_linear_analitica.ipynb

kind            x            y          w0         w1        R2  \
0   analytical  Shoe number       Height   10.121651   4.053374  0.710977   
1   analytical       Height  Shoe number    9.639347   0.175404  0.710977   
2   analytical  Shoe number          Sex   -3.485924   0.105604  0.466478   
3   analytical          Sex  Shoe number   36.469147   4.417225  0.466478   
4   analytical  Shoe number       Weight  -60.230206   3.307012  0.380260   
5   analytical       Weight  Shoe number   31.401739   0.114986  0.380260   
6   analytical       Weight       Height  134.042145   0.513859  0.328625   
7   analytical       Height       Weight  -38.473419   0.639523  0.328625   
8   analytical       Height          Sex   -2.002969   0.015791  0.241039   
9   analytical          Sex       Height  159.753143  15.263947  0.241039   
10  analytical       Weight          Sex   -0.056145   0.010529  0.133350   
11  analytical          Sex       Weight   61.703724  12.665610  0.133350   
12  analytical          Age          Sex    0.899091  -0.007524  0.046283   
13  analytical          Sex          Age   32.691368  -6.151586  0.046283   
14  analytical       Weight          Age   17.666733   0.153627  0.034725   

           MSE           SSE           SSR           SST    N    wall_s  \
0    60.298985  15496.838867  38121.156250  53618.070312  257  0.000208   
1     2.609349    670.602661   1649.638916   2320.241211  257  0.000207   
2     0.115155     29.594906     25.875904     55.470818  257  0.000304   
3     4.816732   1237.900146   1082.339722   2320.241211  257  0.000140   
4   160.916199  41355.464844  25374.904297  66730.390625  257  0.000182   
5     5.595119   1437.945435    882.296143   2320.241211  257  0.000564   
6   140.069473  35997.855469  17620.212891  53618.070312  257  0.001042   
7   174.323517  44801.144531  21929.246094  66730.390625  257  0.000178   
8     0.163814     42.100174     13.370642     55.470818  257  0.000112   
9   158.342453  40694.011719  12924.040039  53618.070312  257  0.000358   
10    0.187057     48.073776      7.397038     55.470818  257  0.000288   
11  225.026794  57831.886719   8898.496094  66730.390625  257  0.000273   
12    0.205850     52.903461      2.567356     55.470818  257  0.000139   
13  168.307419  43255.007812   2099.127930  45354.128906  257  0.001609   
14  170.347122  43779.210938   1574.917603  45354.128906  257  0.000124   

       peak_mb iters grad_norm gap_w      cond(X)  
0   105.609375  None      None  None   522.453308  
1   105.375000  None      None  None  2020.195068  
2   105.578125  None      None  None   522.453308  
3   105.203125  None      None  None     3.325836  
4   105.609375  None      None  None   522.453308  
5   105.578125  None      None  None   323.549805  
6   105.437500  None      None  None   323.549805  
7   105.375000  None      None  None  2020.195068  
8   105.265625  None      None  None  2020.195068  
9   105.109375  None      None  None     3.325836  
10  105.375000  None      None  None   323.549805  
11  105.171875  None      None  None     3.325836  
12  105.203125  None      None  None    74.397644  
13  104.484375  None      None  None     3.325836  
14  105.390625  None      None  None   323.549805

[INFO] Para plotar execute usando a opção (args.plot = True).


### Regressão Linear Interativa: Batch Gredient Descent

In [83]:
from types import SimpleNamespace

args = SimpleNamespace()
args.plot = False
args.top_k = 15

In [86]:
%run regressao_linear_interativa_BGD.ipynb 

kind            x            y          w0         w1        R2  \
0   BGD          Sex          Age   32.691303  -6.151573  0.046283   
1   BGD          Sex       Height  159.752731  15.263940  0.241039   
2   BGD          Sex       Weight   61.703541  12.665589  0.133350   
3   BGD          Sex  Shoe number   36.469051   4.417215  0.466478   
4   BGD          Age          Sex    0.899089  -0.007524  0.046283   
5   BGD          Age       Height  169.239090   0.033947  0.000975   
6   BGD          Age       Weight   63.940125   0.226034  0.034725   
7   BGD          Age  Shoe number   39.727516  -0.008197  0.001314   
8   BGD       Height          Sex   -2.002966   0.015791  0.241039   
9   BGD       Height          Age   23.591116   0.028715  0.000975   
10  BGD       Height       Weight  -38.473297   0.639522  0.328625   
11  BGD       Height  Shoe number    9.639334   0.175403  0.710977   
12  BGD       Weight          Sex   -0.056145   0.010528  0.133350   
13  BGD       Weight          Age   17.666719   0.153626  0.034725   
14  BGD       Weight       Height  134.041901   0.513857  0.328625   
15  BGD       Weight  Shoe number   31.401661   0.114986  0.380260   
16  BGD  Shoe number          Sex   -3.485917   0.105604  0.466478   
17  BGD  Shoe number          Age   34.806862  -0.160234  0.001313   
18  BGD  Shoe number       Height   10.121475   4.053368  0.710977   
19  BGD  Shoe number       Weight  -60.230133   3.307004  0.380260   

           MSE  iters  grad_norm    wall_s  peak_mb    N  
0   168.307419   8000   0.000094  0.403593      0.0  257  
1   158.342468   8000   0.000728  0.418197      0.0  257  
2   225.026779   8000   0.000357  0.456383      0.0  257  
3     4.816731   8000   0.000184  0.403296      0.0  257  
4     0.205850   8000   0.000003  0.397686      0.0  257  
5   208.427261   8000   0.000737  0.400810      0.0  257  
6   250.634933   8000   0.000378  0.396792      0.0  257  
7     9.016316   8000   0.000190  0.394656      0.0  257  
8     0.163814   8000   0.000003  0.398594      0.0  257  
9   176.303192   8000   0.000095  0.397941      0.0  257  
10  174.323517   8000   0.000376  0.397066      0.0  257  
11    2.609349   8000   0.000190  0.397104      0.0  257  
12    0.187057   8000   0.000003  0.397218      0.0  257  
13  170.347137   8000   0.000092  0.400952      0.0  257  
14  140.069473   8000   0.000743  0.405100      0.0  257  
15    5.595118   8000   0.000184  0.418517      0.0  257  
16    0.115155   8000   0.000003  0.412219      0.0  257  
17  176.243439   8000   0.000095  0.418352      0.0  257  
18   60.299000   8000   0.000741  0.413475      0.0  257  
19  160.916183   8000   0.000382  0.411743      0.0  257

### Regressão Linear Interativa: Stocastic Gradient Descent

In [82]:
from types import SimpleNamespace

args = SimpleNamespace()
args.plot = False
args.top_k = 15

In [ ]:
%run regressao_linear_interativa_SGD.ipynb